
# Zidio Internship Project: Time Series Stock Forecasting
This Jupyter Notebook implements **time series forecasting** for TCS, Apple, Reliance, and Infosys stock prices using LSTM models. Each step is explained and includes outputs (plots, model accuracy) ready for report use.


In [1]:

# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import yfinance as yf


: 

In [ ]:

# Load Stock Data
stocks = ['TCS.NS', 'AAPL', 'RELIANCE.NS', 'INFY.NS']
data = {stock: yf.download(stock, start='2015-01-01', end='2025-09-29') for stock in stocks}


In [ ]:

# Exploratory Data Analysis (EDA)
for stock in stocks:
    plt.figure(figsize=(10,4))
    plt.plot(data[stock]['Close'])
    plt.title(f'{stock} Closing Price')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.show()


In [ ]:

# Data Preprocessing: Scaling
scalers = {}
scaled_data = {}

for stock in stocks:
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data[stock] = scaler.fit_transform(data[stock]['Close'].values.reshape(-1,1))
    scalers[stock] = scaler


In [ ]:

# Function to create dataset for LSTM
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(time_step, len(dataset)):
        X.append(dataset[i-time_step:i, 0])
        y.append(dataset[i,0])
    return np.array(X), np.array(y)

time_step = 60
train_data = {}
test_data = {}
X_train = {}
y_train = {}
X_test = {}
y_test = {}

for stock in stocks:
    dataset = scaled_data[stock]
    training_size = int(len(dataset)*0.8)
    train_data[stock] = dataset[:training_size]
    test_data[stock] = dataset[training_size:]
    
    X_train[stock], y_train[stock] = create_dataset(train_data[stock], time_step)
    X_test[stock], y_test[stock] = create_dataset(test_data[stock], time_step)
    
    X_train[stock] = X_train[stock].reshape(X_train[stock].shape[0], X_train[stock].shape[1], 1)
    X_test[stock] = X_test[stock].reshape(X_test[stock].shape[0], X_test[stock].shape[1], 1)


In [ ]:

# Build & Train LSTM Model
models = {}

for stock in stocks:
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step,1)))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    model.fit(X_train[stock], y_train[stock], epochs=10, batch_size=32, verbose=1)
    models[stock] = model


In [ ]:

# Make Predictions
predictions = {}
for stock in stocks:
    pred = models[stock].predict(X_test[stock])
    pred = scalers[stock].inverse_transform(pred)
    predictions[stock] = pred


In [ ]:

# Plot Actual vs Predicted Prices
for stock in stocks:
    plt.figure(figsize=(10,4))
    plt.plot(data[stock]['Close'][-len(predictions[stock]):], label='Actual')
    plt.plot(predictions[stock], label='Predicted')
    plt.title(f'{stock} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()


In [ ]:

# Evaluate Model
for stock in stocks:
    rmse = np.sqrt(mean_squared_error(data[stock]['Close'][-len(predictions[stock]):], predictions[stock]))
    print(f'{stock} RMSE: {rmse}')
